# TF-IDF 구하기
- 벡터화하기 전에 데이터 전처리
   - document별 형태소 분석
   - 미등록단어 처리
   - 명사만 추출
   - 불용어(stopwords) 처리
- document별 단어로 tokenize되어 있는 것을 다시 합쳐서 각각 하나의 document로 만들기
- 벡터화(vectorize) 하기
- tf-idf 값 구하기

In [1]:
import pandas as pd

In [2]:
# CSV 데이터 불러오기
clova_df = pd.read_csv("../../data/sentiment/sentiment_clova.csv", index_col=0)
kakao_df = pd.read_csv("../../data/sentiment/sentiment_mini.csv", index_col=0)
geni_df = pd.read_csv("../../data/sentiment/sentiment_gigagenie.csv", index_col=0)
nugu_df = pd.read_csv("../../data/sentiment/sentiment_nugu.csv", index_col=0)

In [7]:
# 데이터 합치기
# 긍정 document
pos_c_text = clova_df.positive_text[clova_df.positive_text.notnull()]
pos_k_text = kakao_df.positive_text[kakao_df.positive_text.notnull()]
pos_g_text = geni_df.positive_text[geni_df.positive_text.notnull()]
pos_n_text = nugu_df.positive_text[nugu_df.positive_text.notnull()]

In [8]:
# 데이터 합치기
# 부정 document
neg_c_text = clova_df.negative_text[clova_df.negative_text.notnull()]
neg_k_text = kakao_df.negative_text[kakao_df.negative_text.notnull()]
neg_g_text = geni_df.negative_text[geni_df.negative_text.notnull()]
neg_n_text = nugu_df.negative_text[nugu_df.negative_text.notnull()]

In [9]:
clova_text = [pos_c_text, neg_c_text]
kakao_text = [pos_k_text, neg_k_text]
geni_text = [pos_g_text, neg_g_text]
nugu_text = [pos_n_text, neg_n_text]

In [10]:
len(clova_text), len(kakao_text), len(geni_text), len(nugu_text)

(2, 2, 2, 2)

In [ ]:
'blog.naver.com/ppang7942/'로 시작하는 url 페이지의 데이터 거르기

# <br>
## 클로바

<br/><br/>
### POS tagging하기

#### 1.  Komoran 사용

In [11]:
from konlpy.tag import Komoran

In [12]:
komoran = Komoran(userdic='./unregistered.txt')

In [20]:
import re

# 명사와 외래어만 뽑아내기위한 정규표현식
p = re.compile('NN.*|SL')

In [ ]:
results = []
for i, brand_text in enumerate(clova_text):
    pos_tagging_list = list(map(komoran.pos, brand_text))
    print("pos tagging -", i)

    docs = []
    for doc in pos_tagging_list:
        want_words = [word for word, pos in doc if p.match(pos)]         
        docs.append(want_words)
    print("noun extract -", i)
    
    results.append(sum(docs, []))
    print("finish -", i)

### pickle로 일단 저장

In [ ]:
import pickle
pickle.dump(results, open('tfidf_clova_list.pickle', "wb"))

In [ ]:
load_result = pickle.load(open('tfidf_cloav_list.pickle', "rb"))
results = load_result

<br/><br/>
### stopword 제거

In [ ]:
stopwords = [line.strip() for line in open('./stopwordsKor.txt', encoding='utf-8')]

In [ ]:
from tqdm import tqdm_notebook
stopword_dict = {}
for s_word in stopwords:
    stopword_dict[s_word] = 1

final_docs = []
for i, doc in enumerate(results):
    unique_NN_words = set(doc)
    final_NN_words = doc
    
    for word in tqdm_notebook(unique_NN_words):
        if stopword_dict.get(word):
            final_NN_words = list(filter(lambda x: x!= word, final_NN_words))
    final_docs.append(final_NN_words)
    print("finish -", i)

<br/><br/>
### document별 단어들을 각각의 document로 바꾸기

In [ ]:
documents = []
for doc_words in final_docs:
    document = " ".join(doc_words)
    documents.append(document)

In [ ]:
documents[0][:200]

<br/><br/>
### 벡터화 하기

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer     # 벡터 수 카운트 할 때

In [ ]:
cv=CountVectorizer(max_df=0.85, max_features=10000)     
# 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW(Bag Of Words) 인코딩한 벡터를 만든다.
# max_df=0.85란 말은 documents들에서 85%이상 나타나는 토큰(단어)를 무시하라는 것, 
# max_df=25란 말은 documents들에서 25번 이상 나타나는 토큰(단어)를 무시하라는 것,
# maximum 단어의 개수를 1만개로 정함 - 빈도수가 높은 단어 순으로 1만개를 자름(메모리 에러 안나려고)
# min_df=1, ngram_range=(1,1) 등은 default 값임  

word_count_vector=cv.fit_transform(documents)      # vectorize된 word_count_vector
feature_names=cv.get_feature_names()

In [ ]:
len(cv.vocabulary_)      # 위에서 max_features를 10000으로 주었지만 단어 수가 그 이하였음

In [ ]:
cv.vocabulary_

<br/><br/>
### TF-IDF 적용
- TF * IDF는 특정 문서 내에서 단어 빈도가 높을수록, 전체 문서들에는 그 단어를 포함한 문서가 적을수록 TF * IDF값이 높아지는 특징이 있다.
- 이러한 특징을 이용해서 모든 문서에 나타나는 흔한 단어들을 걸러내며, 특정 단어가 가지는 중요도를 측정하는 데 사용된다.
- 한마디로 TF * IDF 값은 특정 단어가 가지는 중요도!
- TF(Term Frequency): 단어 빈도
   - 해당 문서에서 단어가 나타나는 빈도수
   - 문서의 길이가 길면 해당 단어의 실제 중요도와는 상관없이 단어의 빈도수는 증가될 확률이 높다.
   - 위의 문제를 해결하기 위해 다음과 같이 표준화 -> (문서에서 단어가 나타나는 빈도수) / (모든 단어가 나타나는 빈도수)
- IDF(Inverse Document Frequency): 역문헌 빈도
   - 해당 단어의 일반적인 중요도를 나타내는 값
   - log( 전체 문서의 수 / 해당 단어가 포함된 문서들의 수 )

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer    # Tf * idf 구할 때

In [ ]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)        # Tf-idf 가중치를 적용할 수 있도록 변환시켜줌
tf_idf_matrix = tfidf_transformer.fit_transform(word_count_vector)

feature_names=cv.get_feature_names()

dense = tf_idf_matrix.todense()

for i in range(len(dense)):
    doc = dense[i].tolist()[0]           # dense[i].tolist()는 2차원 list, 예를들면 shape: (1,814)
    phrase_scores = [pair for pair in zip(range(0, len(doc)), doc) if pair[1] > 0]      # 0의 값이 아닌것만 모아서 만듦

    sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1], reverse=True) # sorted(phrase_scores, key=lambda t: t[1] * -1)라고 해도 됨
    for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:20]:
        print('{0: <20} {1}'.format(phrase, score))      # 단어와 단어의 tf-idf 값을 출력
    print()
    ##### 이 부분은 테스트를 위해서 #####
    if i == 3:     
        break
    ###############################

# <br>
## 카카오

<br/><br/>
### POS tagging하기

#### 1.  Komoran 사용

In [87]:
from konlpy.tag import Komoran

In [107]:
komoran = Komoran(userdic='./unregistered.txt')

In [108]:
import re

# 명사와 외래어만 뽑아내기위한 정규표현식
p = re.compile('NN.*|SL')

In [150]:
results = []
for i, brand_text in enumerate(kakao_text):
    pos_tagging_list = list(map(komoran.pos, brand_text))
    print("pos tagging -", i)

    docs = []
    for doc in pos_tagging_list:
        want_words = [word for word, pos in doc if p.match(pos)]         
        docs.append(want_words)
    print("noun extract -", i)
    
    results.append(sum(docs, []))
    print("finish -", i)

pos tagging - 0
noun extract - 0
finish - 0
pos tagging - 1
noun extract - 1
finish - 1
pos tagging - 2
noun extract - 2
finish - 2
pos tagging - 3
noun extract - 3
finish - 3


### pickle로 일단 저장

In [151]:
import pickle
pickle.dump(results, open('tfidf_kakao_list.pickle', "wb"))

In [152]:
load_result2 = pickle.load(open('tfidf_kakao_list.pickle', "rb"))
results = load_result2

<br/><br/>
### stopword 제거

In [153]:
stopwords = [line.strip() for line in open('./stopwordsKor.txt', encoding='utf-8')]

In [154]:
from tqdm import tqdm_notebook
stopword_dict = {}
for s_word in stopwords:
    stopword_dict[s_word] = 1

final_docs = []
for i, doc in enumerate(results):
    unique_NN_words = set(doc)
    final_NN_words = doc
    
    for word in tqdm_notebook(unique_NN_words):
        if stopword_dict.get(word):
            final_NN_words = list(filter(lambda x: x!= word, final_NN_words))
    final_docs.append(final_NN_words)
    #print("finish -", i)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


<br/><br/>
### document별 단어들을 각각의 document로 바꾸기

In [155]:
documents = []
for doc_words in final_docs:
    document = " ".join(doc_words)
    documents.append(document)

In [156]:
documents[0][:200]

'유선 래 연결 와이파이 연결 사용 음성 인식 스피커 일상 차지 역할 흐름 가운데 국내 업체 세계 시장 맥 지적 전문가 국내 업체 시장 확대 이유 길 지적 글 사기 기본 사례 네이버 카카오 날 하루 수준 카메라 인식 로그인 질문 마찬가지 대답 네이버 음악 듣기 결제 상태 지원 별도 이퀄라이저 설정 지하철역 자동차 추천 경로 검색 비교 효과 결과 근처 분식집 '

<br/><br/>
### 벡터화 하기

In [157]:
from sklearn.feature_extraction.text import CountVectorizer     # 벡터 수 카운트 할 때

In [158]:
cv=CountVectorizer(max_df=0.85, max_features=10000)     
# 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW(Bag Of Words) 인코딩한 벡터를 만든다.
# max_df=0.85란 말은 documents들에서 85%이상 나타나는 토큰(단어)를 무시하라는 것, 
# max_df=25란 말은 documents들에서 25번 이상 나타나는 토큰(단어)를 무시하라는 것,
# maximum 단어의 개수를 1만개로 정함 - 빈도수가 높은 단어 순으로 1만개를 자름(메모리 에러 안나려고)
# min_df=1, ngram_range=(1,1) 등은 default 값임  

word_count_vector=cv.fit_transform(documents)      # vectorize된 word_count_vector
feature_names=cv.get_feature_names()

In [159]:
len(cv.vocabulary_)      # 위에서 max_features를 10000으로 주었지만 단어 수가 그 이하였음

9954

In [160]:
cv.vocabulary_

{'듣기': 2969,
 '이퀄라이저': 7067,
 '지하철역': 8068,
 '마인': 3315,
 '자비스': 7277,
 '중반': 7943,
 '실현': 5650,
 '생태계': 4824,
 '번성': 4068,
 '도태': 2824,
 '시범': 5473,
 '여원': 6162,
 '격화': 1447,
 '조짐': 7798,
 '진출': 8122,
 '세컨드': 5025,
 '메신저': 3483,
 '계좌': 1530,
 '개설': 1335,
 '이모티콘': 6968,
 '곳도': 1659,
 '린지': 3254,
 '전만': 7533,
 '경쟁자': 1496,
 '중립': 7940,
 '선상': 4909,
 '생물': 4806,
 'ceo': 134,
 '주년': 7847,
 '소회': 5120,
 '시너지': 5449,
 '내부': 2324,
 '논의': 2421,
 '액션': 5927,
 '완결': 6476,
 '베스트': 4108,
 '브레인': 4435,
 '노동부': 2378,
 '권리': 1922,
 '다라': 2511,
 '베타서비스': 4118,
 '제지': 7749,
 '하루하루': 9406,
 '실무': 5624,
 '시각': 5430,
 '시행착오': 5524,
 '웹툰': 6713,
 '회화': 9883,
 '감안': 1259,
 '매니저': 3410,
 '역삼동': 6184,
 '데모': 2750,
 '세션': 5014,
 '테크니컬': 8897,
 '전산언어학': 7544,
 '자인': 7299,
 '최현정': 8430,
 '연구원': 6203,
 '킹스': 8791,
 '골든': 1643,
 '서클': 4878,
 '범죄도시': 4084,
 '제품군': 7751,
 '만회': 3367,
 '라인업': 3067,
 '악의': 5811,
 '점수': 7597,
 '필터링': 9379,
 '진미': 8104,
 '열흘': 6279,
 'amp': 25,
 '상거래': 4728,
 '전환율': 7583,
 '격차': 1446,


<br/><br/>
### TF-IDF 적용
- TF * IDF는 특정 문서 내에서 단어 빈도가 높을수록, 전체 문서들에는 그 단어를 포함한 문서가 적을수록 TF * IDF값이 높아지는 특징이 있다.
- 이러한 특징을 이용해서 모든 문서에 나타나는 흔한 단어들을 걸러내며, 특정 단어가 가지는 중요도를 측정하는 데 사용된다.
- 한마디로 TF * IDF 값은 특정 단어가 가지는 중요도!
- TF(Term Frequency): 단어 빈도
   - 해당 문서에서 단어가 나타나는 빈도수
   - 문서의 길이가 길면 해당 단어의 실제 중요도와는 상관없이 단어의 빈도수는 증가될 확률이 높다.
   - 위의 문제를 해결하기 위해 다음과 같이 표준화 -> (문서에서 단어가 나타나는 빈도수) / (모든 단어가 나타나는 빈도수)
- IDF(Inverse Document Frequency): 역문헌 빈도
   - 해당 단어의 일반적인 중요도를 나타내는 값
   - log( 전체 문서의 수 / 해당 단어가 포함된 문서들의 수 )

In [161]:
from sklearn.feature_extraction.text import TfidfTransformer    # Tf * idf 구할 때

In [162]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)        # Tf-idf 가중치를 적용할 수 있도록 변환시켜줌
tf_idf_matrix = tfidf_transformer.fit_transform(word_count_vector)

feature_names=cv.get_feature_names()

dense = tf_idf_matrix.todense()

for i in range(len(dense)):
    doc = dense[i].tolist()[0]           # dense[i].tolist()는 2차원 list, 예를들면 shape: (1,814)
    phrase_scores = [pair for pair in zip(range(0, len(doc)), doc) if pair[1] > 0]      # 0의 값이 아닌것만 모아서 만듦

    sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1], reverse=True) # sorted(phrase_scores, key=lambda t: t[1] * -1)라고 해도 됨
    for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:20]:
        print('{0: <20} {1}'.format(phrase, score))      # 단어와 단어의 tf-idf 값을 출력
    print()
    ##### 이 부분은 테스트를 위해서 #####
    if i == 3:     
        break
    ###############################

out                  0.2343521109627114
미니언즈                 0.12819475081624657
펌웨어                  0.12819475081624657
wave                 0.12034297589977073
nonnull              0.11247143563490924
엘지                   0.09230022058769753
반려견                  0.08837041371314297
약관                   0.08717243055504767
보호자                  0.08233993087879049
float                0.07230306576529881
목적어                  0.07230306576529881
와서                   0.07178906045709808
후에                   0.07178906045709808
셀리                   0.06967224920513042
필립스                  0.06666127042444822
for                  0.06426939179137671
int                  0.06426939179137671
parse                0.06426939179137671
선풍기                  0.06333840836830039
잠시                   0.06333840836830039

취약점                  0.12963442266623498
보험료                  0.12454112383263523
선생님                  0.10082677318484944
크림                   0.09785374015421339
수영              